# Neural network working as a decoder (decodes a binary sequence to the number)
## Example: [1,0,0,0,1,0,0,1] -> 137

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

## Create the dataset
- input: lists of size 8 with 0 and 1
- output: the value of the number

In [2]:
samples = []
labels = []
for x in range(256):
    bin_val = bin(x)[2:].rjust(8,'0')
    samples.append(list(bin_val))
    labels.append(x)
samples = np.array(samples,dtype=float)
labels = np.array(labels,dtype=float)

for i in range(5):
    print(samples[i],' -> ',labels[i])    
print('...')
for i in range(250,256):
    print(samples[i],' -> ',labels[i])

[0. 0. 0. 0. 0. 0. 0. 0.]  ->  0.0
[0. 0. 0. 0. 0. 0. 0. 1.]  ->  1.0
[0. 0. 0. 0. 0. 0. 1. 0.]  ->  2.0
[0. 0. 0. 0. 0. 0. 1. 1.]  ->  3.0
[0. 0. 0. 0. 0. 1. 0. 0.]  ->  4.0
...
[1. 1. 1. 1. 1. 0. 1. 0.]  ->  250.0
[1. 1. 1. 1. 1. 0. 1. 1.]  ->  251.0
[1. 1. 1. 1. 1. 1. 0. 0.]  ->  252.0
[1. 1. 1. 1. 1. 1. 0. 1.]  ->  253.0
[1. 1. 1. 1. 1. 1. 1. 0.]  ->  254.0
[1. 1. 1. 1. 1. 1. 1. 1.]  ->  255.0


## Build the neural network model

In [3]:
ADDITIONAL_LAYER = True

model = Sequential()
model.add(Dense(12,input_dim=8))
if ADDITIONAL_LAYER:
    model.add(Dense(12))
model.add(Dense(1))
model.summary()
num_epochs = 0

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                108       
_________________________________________________________________
dense_1 (Dense)              (None, 12)                156       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 13        
Total params: 277
Trainable params: 277
Non-trainable params: 0
_________________________________________________________________


## Compile the model

In [4]:
model.compile(loss='mean_absolute_error', optimizer="adam",metrics=['mae'])

## Divide to training and test 

In [5]:
TRAIN_SIZE = 0.1 # only 10% to train!

import sklearn.model_selection
(trainSamples, testSamples, trainLabels, testLabels) = sklearn.model_selection.train_test_split(samples, labels, 
                            train_size=TRAIN_SIZE)
print('Training samples:',len(trainSamples),' test samples:',len(testSamples))

Training samples: 25  test samples: 231


## Train the network

In [13]:
EPOCHS = 200
print("Training using ",len(trainSamples),'samples and ',EPOCHS,' epochs')

H = model.fit(trainSamples,trainLabels,validation_data=(testSamples,testLabels), epochs=EPOCHS, verbose=0)
num_epochs+=EPOCHS
print('Epochs',num_epochs)
print("Initial loss - training mae={:6.3f} validation mae={:6.3f}".format(H.history['mae'][0],H.history['val_mae'][0]))
print("Final loss - training mae={:6.3f} validation mae={:6.3f}".format(H.history['mae'][-1],H.history['val_mae'][-1]))

Training using  25 samples and  200  epochs
Epochs 1020
Initial loss - training mae= 1.049 validation mae= 2.149
Final loss - training mae= 0.073 validation mae= 0.066


## Calculate errors

In [14]:
def calc_errors(predictedLabels,realLabels,verbose=1):
    predictedLabels = np.ravel(predictedLabels)
    for i in range(len(realLabels)):
        tNum = realLabels[i]
        pNum = predictedLabels[i]
        if(verbose>0):
            print("{:3.0f} -> {:6.2f} error={:7.2f}".format(tNum,pNum,abs(pNum-tNum)))
    print('Mean Absolute Error (MAE)= {:7.2f}'.format(np.mean(np.absolute(predictedLabels - realLabels))))
print('TRAINING SET')
predictedLabels = model.predict(trainSamples)
calc_errors(predictedLabels,trainLabels)
print()

print('TEST SET')
predictedLabels = model.predict(testSamples)
calc_errors(predictedLabels,testLabels)


TRAINING SET
 85 ->  85.00 error=   0.00
129 -> 128.94 error=   0.06
190 -> 189.89 error=   0.11
205 -> 204.92 error=   0.08
 35 ->  34.95 error=   0.05
 27 ->  26.97 error=   0.03
249 -> 248.90 error=   0.10
 76 ->  75.97 error=   0.03
143 -> 142.91 error=   0.09
139 -> 138.90 error=   0.10
236 -> 235.89 error=   0.11
 28 ->  27.99 error=   0.01
 15 ->  14.97 error=   0.03
 47 ->  46.94 error=   0.06
239 -> 238.88 error=   0.12
 20 ->  20.00 error=   0.00
133 -> 132.95 error=   0.05
144 -> 143.94 error=   0.06
 16 ->  15.99 error=   0.01
 97 ->  96.95 error=   0.05
 22 ->  21.98 error=   0.02
 46 ->  45.93 error=   0.07
182 -> 181.91 error=   0.09
135 -> 134.93 error=   0.07
 54 ->  53.96 error=   0.04
Mean Absolute Error (MAE)=    0.06

TEST SET
234 -> 233.86 error=   0.14
 91 ->  90.95 error=   0.05
 69 ->  68.99 error=   0.01
  1 ->   0.99 error=   0.01
228 -> 227.90 error=   0.10
130 -> 129.91 error=   0.09
 21 ->  21.01 error=   0.01
117 -> 116.98 error=   0.02
 39 ->  38.96 erro